**Einlesen der Daten**

In [ ]:
import pandas as pd

df = pd.read_csv("trainingsdaten_ohne_vibration.csv", encoding = "ISO-8859-1")
df_test = pd.read_csv("testdaten_mit_vibration.csv", encoding = "ISO-8859-1")

**Trainingsdaten anschauen**

In [ ]:
df.head()

**Testdaten anschauen**

In [ ]:
df_test.head()

** *Time* löschen**

In [ ]:
del df["Time"]
del df_test["Time"]

**Deskriptive Statistik (Trainingsdaten)**

In [ ]:
df.describe()

**Deskriptive Statistik (Testdaten)**

In [ ]:
df_test.describe()

**Visualize data**

In [ ]:
import matplotlib.pyplot as plt

df.plot(subplots=True, layout=(3,2), legend=False)
plt.show()

In [ ]:
df_test.plot(subplots=True, layout=(3,2), legend=False)
plt.show()

**Datenvorverarbeitung**

In [ ]:
# 2 Spalten enthalten keine Information -> Löschen
df = df.drop(['NumberOverload', 'NumberUnderloads'], 1)
df_test = df_test.drop(['NumberOverload', 'NumberUnderloads'], 1)

**Test- und Trainingsdaten festlegen**

In [ ]:
X_train = df
X_test = df_test[['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].copy()
y_test = df_test[['VibrationMotorOn']].copy()

**Modell trainieren**

In [ ]:
from sklearn.ensemble import IsolationForest

# Modell fitten
clf = IsolationForest()
clf.fit(X_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

new_column = pd.DataFrame(columns = ["Anomaly"])
# Vorhersage des Modells
for k in range(len(X_test)):
    temp = clf.predict([[X_test["AmplitudeBandWidth"][k], X_test["AmplitudeMean"][k], X_test["StabilizationTime"][k]]])[0]
    new_column = new_column.append({"Anomaly": temp}, ignore_index = True)

# Vorhersage des Modells auf 1/0 recodieren
new_column['Anomaly'] = new_column['Anomaly'].map( {1: 0, -1: 1} )

**Metriken ausgeben**

In [ ]:
from sklearn import metrics

print("Accuracy: ", metrics.accuracy_score(y_test, new_column))  
print("Precision: ", metrics.precision_score(y_test, new_column))  
print("Recall: ", metrics.recall_score(y_test, new_column))  
print("F1: ", metrics.f1_score(y_test, new_column))  
print("Confusion Matrix:")
print(metrics.confusion_matrix(y_test, new_column))  

**Daten zusammenführen für Visualisierung**

In [ ]:
complete_data = X_test.copy()
complete_data['VibrationMotorOn'] = y_test['VibrationMotorOn']
complete_data['Anomaly'] = new_column.values

**Visualisieren der Vorhersagen**

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots()
a = complete_data[['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].loc[complete_data['Anomaly'] == 1] # Anomalie vorhergesagt
n = complete_data[['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].loc[complete_data['Anomaly'] == 0] # Normalwert vorhergesagt

ax.scatter(a.index,a.mean(axis=1), color='red', label='Anomalie vorhergesagt')
ax.scatter(n.index,n.mean(axis=1), color='green', label='Normalwert vorhergesagt')

plt.legend()
plt.show()

**Visualisieren der tatsächlichen Anomalien**

In [ ]:
fig, ax = plt.subplots()
a = complete_data[['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].loc[complete_data['VibrationMotorOn'] == 1] #true anomaly
n = complete_data[['AmplitudeBandWidth', 'AmplitudeMean', 'StabilizationTime']].loc[complete_data['VibrationMotorOn'] == 0] #true normal

ax.scatter(a.index,a.mean(axis=1), color='red', label='Tatsaechliche Anomalie')
ax.scatter(n.index,n.mean(axis=1), color='green', label='Tatsaechlicher Normalwert')

plt.legend()
plt.show()